In [1]:
%matplotlib widget

import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt

from ProcessResults import *
from DatasetTester import *
from GetLandmarks import GetLandmarks 
#from LandmarkTester_alt import LandmarkTester

from landmark_utils import vis_landmarks, compare_landmarks
from vis_utils import *


import cv2
import numpy as np
import time
import timeit
from pathlib import Path
import csv
import pickle

In [2]:
MEEI_path = Path('./MEEI_Standard_Set')
pat_info = pd.read_excel(MEEI_path/"pat_info.xlsx", sheet_name='Sheet1')

In [3]:
#pat_info

In [4]:
def get_path(cat, subcat, num):
    if cat == 'Normal':
        return Path(MEEI_path/cat/(cat+str(num)))
    return Path(MEEI_path/cat/(subcat+cat)/(subcat+cat+str(num)))

def get_hb(cat, sub):
    if cat == 'Normal':
        return 1
    elif sub == 'NearNormal':
        return 2
    elif sub == 'Mild':
        return 3
    elif sub == 'Moderate':
        return 4
    elif sub == 'Severe':
        return 5
    elif sub == 'Complete':
        return 6
    else:
        return 0

In [5]:
pat_info['Path'] = pat_info.apply(lambda row: get_path(row['Category'],row['Sub-category'],row['#']), axis=1)
pat_info['HB_scale'] = pat_info.apply(lambda row: get_hb(row['Category'],row['Sub-category']), axis=1)
pat_info

,Category,Sub-category,#,Side,Gender,Age,Path,HB_scale
0,Flaccid,Complete,1,Left,Male,54,MEEI_Standard_Set/Flaccid/CompleteFlaccid/Comp...,6
1,Flaccid,Complete,2,Right,Female,16,MEEI_Standard_Set/Flaccid/CompleteFlaccid/Comp...,6
2,Flaccid,Complete,3,Right,Male,38,MEEI_Standard_Set/Flaccid/CompleteFlaccid/Comp...,6
3,Flaccid,Complete,4,Left,Female,31,MEEI_Standard_Set/Flaccid/CompleteFlaccid/Comp...,6
4,Flaccid,Complete,5,Left,Female,52,MEEI_Standard_Set/Flaccid/CompleteFlaccid/Comp...,6
5,Flaccid,Mild,1,Left,Male,72,MEEI_Standard_Set/Flaccid/MildFlaccid/MildFlac...,3
6,Flaccid,Mild,2,Right,Female,55,MEEI_Standard_Set/Flaccid/MildFlaccid/MildFlac...,3
7,Flaccid,Mild,3,Left,Male,75,MEEI_Standard_Set/Flaccid/MildFlaccid/MildFlac...,3
8,Flaccid,Moderate,1,Left,Male,51,MEEI_Standard_Set/Flaccid/ModerateFlaccid/Mode...,4
9,Flaccid,Moderate,3,Left,Female,62,MEEI_Standard_Set/Flaccid/ModerateFlaccid/Mode...,4


In [6]:
# Store the table to a pickle file
pat_info.to_pickle('MEEI_pat_info.pkl')

In [7]:
class MEEI_Patient:
    def __init__(self, id, pat_info_row):
        super().__init__()
        self._id = id
        self._path = pat_info_row['Path']
        print(self._path)
        # Could potentially also have all the info from pat_info_row here, i.e. side, age, gender, etc.
        
        self._img = np.zeros((8,1080,720,3),dtype=np.uint8) # could potentially use frames from video in the future
        self._landmarks = np.zeros((8,68,2),dtype=np.int16) # only using the default model for now
        #self._dists = np.zeros((8,68,2),dtype=np.int16)
        self._features = np.zeros((8,29),dtype=np.float32)
        self._hb_scale = [pat_info_row['HB_scale'] for i in range(8)]
        
        self.load_imgs()
        self.get_landmarks_features()

    def load_imgs(self):
        img_paths = []
        for x in self._path.iterdir():
            if x.is_file():
                if x.suffix == '.jpg' or x.suffix == '.JPG':
                    img_paths.append(x)
                
        for i in range(len(img_paths)):
            im = cv2.imread(str(img_paths[i]))
            if im.shape[0] != 1080 or im.shape[1] != 720:
                im = cv2.resize(im,(720,1080))
            self._img[i] = im
            
    def get_landmarks_features(self):
        for i in range(8):
            img_lm = GetLandmarks(self._img[i])
            self._landmarks[i] = img_lm._shape
            
            lm_results = ProcessResults(img_lm)
            self._features[i] = lm_results._features_arr # 29 features
            

In [8]:
all_pats = np.zeros((len(pat_info)),dtype=object)

for i in range(len(pat_info)):
    all_pats[i] = MEEI_Patient(i, pat_info.loc[i])

MEEI_Standard_Set/Flaccid/CompleteFlaccid/CompleteFlaccid1
Successfully predicted landmarks!


ValueError: could not broadcast input array from shape (51,2) into shape (68,2)

In [ ]:
all_pats[30]._id

30

In [ ]:
all_ids = np.concatenate([np.full((8), i) for i in range(len(pat_info))])
all_imgs = np.concatenate([all_pats[x]._img for x in range(len(all_pats))]) 
all_lms = np.concatenate([all_pats[x]._landmarks for x in range(len(all_pats))]) 
all_features = np.concatenate([all_pats[x]._features for x in range(len(all_pats))]) 
all_hb = np.concatenate([all_pats[x]._hb_scale for x in range(len(all_pats))])

all_pats_processed_data = [all_ids, all_imgs, all_lms, all_features, all_hb]

In [ ]:
# Store the processed data to a pickle file
with open('all_pats_processed_data.pickle', 'wb') as f:
    pickle.dump(all_pats_processed_data, f)

In [ ]:
arr_info(all_ids)

((408,), dtype('int64'), 0, 50)